<a href="https://colab.research.google.com/github/diegobnh/odroid_xu4/blob/master/Selecting_PerformanceCounters_Publica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/mntDrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mntDrive


# Global variables, methods and library

In [0]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from yellowbrick.model_selection import CVScores
from sklearn.metrics import r2_score

import statsmodels.api as sm
import operator
import timeit

import scipy.cluster.hierarchy as shc
from enum import Enum
from scipy.spatial.distance import pdist

from collections import defaultdict



'''
RFR = RANDOM FOREST REGRESSOR
DTR = DECISION TREE REGRESSOR
RFE = RECURSIVE FEATURE ELIMINATION
RFI = RECURSIVE FEATURE INSERT
'''

FILTER_METHOD, FEATURE_IMPORTANCE_DTR, FEATURE_IMPORTANCE_RFR, RFE_LINEAR, RFI_LINEAR, RFI_DTR, RFI_RFR = range(7)

PERFORMANCE_TARGET, POWER_TARGET = range(2)    
    
FOUR_LITTLE, FOUR_BIG, FOUR_LITTLE_FOUR_BIG, FOUR_LITTLE_FOUR_BIG_L, FOUR_LITTLE_FOUR_BIG_B = range(5)


g_number_iterations = 30


#Will remove lines if exist outliers
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

#Will remove columns if exist low std by column
def remove_low_std(file_path,threshold):
    df = pd.read_csv(file_path)     
    columns_to_drop = list(df.loc[:, df.std() < threshold].columns)
    #columns_to_drop.remove('speedup4l')
    print("Num columns to remove:",columns_to_drop)
    #df[columns_to_drop].describe()
    if columns_to_drop.count('speedup4l') > 0 or columns_to_drop.count('speedup4b') > 0 or columns_to_drop.count('speedup4b4l') > 0 : 
      #columns_to_drop.remove('speedup4l')
      print("target column will be removed. !!!!")
    if columns_to_drop.count('power') > 0:    
      print("target column will be removed. !!!!")
    df = df.drop(columns_to_drop, axis=1)
    
    return df

def print_performance(list_scores,list_pmcs):
    print("")
    for i in range(len(list_pmcs)):
       my_list =  list_pmcs[i]
       my_formatted_list = [ '%02d' % elem for elem in my_list ] 
       print("Mean Accuracy: {0:.2f} ".format(round(list_scores[i],2)),my_formatted_list)
    print("Final Mean : {0:.2f} ".format(round(sum(list_scores)/len(list_scores),2)))

def print_selected_pmcs(my_dict, num_pmcs):    
    dictionary = sorted(my_dict.items(), key=operator.itemgetter(1), reverse=True)
    for i in range(num_pmcs):
       string = "\'"+dictionary[i][0]+"\'" 
       print(string, end =",") 
    
   

# Wrapper Method

Wrapper-based methods use models to decide which pmcs to select.

In [0]:
#here cycles is selected in the begin
def select_feature_RFI(file_path, config, type_of_model,type_of_target):

    df = pd.read_csv(file_path)
    for i in df.columns:
       new_df=remove_outlier(df, i)

    if type_of_target == POWER_TARGET:
       array = new_df.values
       target_columns = 1
    elif type_of_target == PERFORMANCE_TARGET:  #Performance we dont call remove_outliers 
       array = df.values
       target_columns = 3

    index_pmcs_selected = []
    best_score = -1
    best_index = -1
    cycle_index = 0
    start_index = 0
    end_index = len(df.columns)-target_columns         

    if config==FOUR_LITTLE:
        total_pmcs = 4
    elif config==FOUR_BIG:  
        total_pmcs = 6
    elif config==FOUR_LITTLE_FOUR_BIG_L:
        total_pmcs = 4
        end_index = 34 #until 33
    elif config==FOUR_LITTLE_FOUR_BIG_B:
        total_pmcs = 6
        if type_of_target == PERFORMANCE_TARGET:
            start_index = cycle_index = 34
    else:
        print("Error config!!")
 
    pmcs_selected = array[:,[cycle_index]] 
    index_pmcs_selected.append(cycle_index)
    for j in range(total_pmcs):        
        for i in range (start_index,end_index): 
            if type_of_model == RFI_LINEAR:
                if type_of_target == PERFORMANCE_TARGET:
                    print("O modelo linear não suporta multitarget!!!")
                else:
                    model = LinearRegression()
            elif type_of_model == RFI_DTR:
                model = DecisionTreeRegressor()             
            elif type_of_model == RFI_RFR:
                model = RandomForestRegressor(n_estimators=50, max_depth=30)
            

            if i not in index_pmcs_selected:                
                column = array[:,[i]] #seleciona a coluna a ser testada    
                X = np.concatenate((pmcs_selected, column), axis=1) #concatena com os atuais pmus
                Y = array[:, (len(df.columns) - target_columns): len(df.columns)]

                #dataset performance is too small, so i need to use all datast
                if type_of_target == PERFORMANCE_TARGET:
                   model.fit(X, Y)                   
                   y_pred = model.predict(X)
                   current_score = r2_score(Y,y_pred)           
                #dataset power is good, so i can use kfold   
                elif type_of_target == POWER_TARGET:    
                   cv = KFold(n_splits=5, shuffle=True)
                   scores = cross_val_score(model, X, Y.ravel(), cv = cv, scoring='r2')                
                   current_score = scores.mean()     
            
                if current_score > best_score:
                    best_score = current_score
                    best_index = i
        if best_index not in index_pmcs_selected:      
            #print("PMU selected:",df.columns.values[best_index]," Score:",best_score)
            index_pmcs_selected.append(best_index)
            column = array[:,[best_index]] 
            pmcs_selected = np.concatenate((pmcs_selected, column), axis=1) 


    string_pmcs_selected = []
    for s in range(len(index_pmcs_selected)):
        string_pmcs_selected.append(df.columns[index_pmcs_selected[s]])
       
    X = array[:,index_pmcs_selected]
    Y = array[:, (len(df.columns) - target_columns): len(df.columns)]

   
    if type_of_target == PERFORMANCE_TARGET:        
        model.fit(X, Y)
        y_pred = model.predict(X)
        score = r2_score(Y,y_pred)
        #score = model.score(X,Y)   
        return score, index_pmcs_selected,string_pmcs_selected
    elif type_of_target == POWER_TARGET:   
        #cv = ShuffleSplit(n_splits=10)
        cv = KFold(n_splits=5, shuffle=True)
        scores = cross_val_score(model, X, Y.ravel(), cv = cv, scoring='r2')
        #visualizer = CVScores(model, cv=cv, scoring='r2')
        #visualizer.fit(X, Y)        # Fit the data to the visualizer
        return scores.mean(),index_pmcs_selected,string_pmcs_selected

# Performance Model

In [0]:
dataset = [("\nResult of 4l - Test==Train","/mntDrive/My Drive/Doutorado/Feature Selection/data/bots/NOT_normalized/4l.csv",FOUR_LITTLE, 5, RFI_RFR, PERFORMANCE_TARGET),
           ("\nResult of 4b - Test==Train","/mntDrive/My Drive/Doutorado/Feature Selection/data/bots/NOT_normalized/4b.csv",FOUR_BIG, 7, RFI_RFR, PERFORMANCE_TARGET),
           ("\nResult of 4b4l_A7 - Test==Train","/mntDrive/My Drive/Doutorado/Feature Selection/data/bots/NOT_normalized/4b4l.csv",FOUR_LITTLE_FOUR_BIG_L, 5, RFI_RFR, PERFORMANCE_TARGET),
           ("\nResult of 4b4l_A15 - Test==Train","/mntDrive/My Drive/Doutorado/Feature Selection/data/bots/NOT_normalized/4b4l.csv",FOUR_LITTLE_FOUR_BIG_B, 7, RFI_RFR, PERFORMANCE_TARGET)
          ]


for out_string,file_name, config, num_pmcs, selection_feature_type, target_type in dataset:
    list_scores = []
    list_index_pmcs = []
    my_dict = {}
    start = timeit.default_timer()
    for i in range(g_number_iterations):
        #print(end = '.')
        score,index_pmcs_selected,string_pmcs_selected  = select_feature_RFI(file_name,config, selection_feature_type,target_type)
        list_scores.append(score)
        list_index_pmcs.append(index_pmcs_selected)

        for j in range(len(index_pmcs_selected)):
            #my_dict[str(string_pmcs_selected[j])] += 1
            if my_dict.__contains__(string_pmcs_selected[j]):
                my_dict[string_pmcs_selected[j]] += 1
            else:
                my_dict[string_pmcs_selected[j]] = 1

    print(out_string)
    print_performance(list_scores,list_index_pmcs)
    print_selected_pmcs(my_dict, num_pmcs)
    print("\nTime(minutes) {0:.2f} ".format(round((timeit.default_timer() - start))/60,2))


Result of 4l - Test==Train

Mean Accuracy: 0.97  ['00', '13', '02']
Mean Accuracy: 0.96  ['00', '13', '19']
Mean Accuracy: 0.97  ['00', '13', '25', '31']
Mean Accuracy: 0.97  ['00', '31', '07', '33', '06']
Mean Accuracy: 0.95  ['00', '02', '31']
Mean Accuracy: 0.97  ['00', '02', '13', '17']
Mean Accuracy: 0.91  ['00', '31', '22']
Mean Accuracy: 0.94  ['00', '13', '30']
Mean Accuracy: 0.97  ['00', '13', '10', '09']
Mean Accuracy: 0.94  ['00', '13', '21', '28']
Mean Accuracy: 0.96  ['00', '13', '33']
Mean Accuracy: 0.96  ['00', '13', '07']
Mean Accuracy: 0.95  ['00', '13', '23', '24']
Mean Accuracy: 0.96  ['00', '13', '02']
Mean Accuracy: 0.97  ['00', '13', '33']
Mean Accuracy: 0.96  ['00', '02', '13']
Mean Accuracy: 0.95  ['00', '13', '30']
Mean Accuracy: 0.96  ['00', '13']
Mean Accuracy: 0.96  ['00', '02', '13', '08']
Mean Accuracy: 0.97  ['00', '31', '13', '07', '32']
Mean Accuracy: 0.97  ['00', '13', '22', '32']
Mean Accuracy: 0.95  ['00', '13', '25', '14']
Mean Accuracy: 0.96  ['00

# Power Model

<img src="http://drive.google.com/uc?export=view&id=1iWjcJ_bdWfHiwAEMU8fqvwQr6wguVcgs">

In [4]:
#Tentando generalizar todos as células abaixo
dataset = [("\n\nResult of 4l -  KFold 5 (score_mean)\n", "/mntDrive/My Drive/Doutorado/Feature Selection/data/power/NOT_normalized/4l.csv",FOUR_LITTLE, 5, RFI_RFR,POWER_TARGET),  
           ("\n\nResult of 4b - KFold 5 (score_mean)\n", "/mntDrive/My Drive/Doutorado/Feature Selection/data/power/NOT_normalized/4b.csv",FOUR_BIG, 7, RFI_RFR,POWER_TARGET),
           ("\n\nResult of 4b4l-A7 - KFold 5 (score_mean)\n", "/mntDrive/My Drive/Doutorado/Feature Selection/data/power/NOT_normalized/4b4l_A7.csv", FOUR_LITTLE_FOUR_BIG_L, 5, RFI_RFR,POWER_TARGET),       
           ("\n\nResult of 4b4l-A15 - KFold 5 (score_mean)\n", "/mntDrive/My Drive/Doutorado/Feature Selection/data/power/NOT_normalized/4b4l_A15.csv",FOUR_LITTLE_FOUR_BIG_B, 7, RFI_RFR,POWER_TARGET)
          ]

for out_string,file_name, config, num_pmcs, selection_feature_type, target_type in dataset:
    list_scores = []
    list_index_pmcs = []
    my_dict = {}
    start = timeit.default_timer()
    for i in range(g_number_iterations):
        #print(end = '.')
        score,index_pmcs_selected,string_pmcs_selected  = select_feature_RFI(file_name,config, selection_feature_type,target_type)
        list_scores.append(score)
        list_index_pmcs.append(index_pmcs_selected)

        for j in range(len(index_pmcs_selected)):
            #my_dict[str(string_pmcs_selected[j])] += 1
            if my_dict.__contains__(string_pmcs_selected[j]):
                my_dict[string_pmcs_selected[j]] += 1
            else:
                my_dict[string_pmcs_selected[j]] = 1

    print(out_string)
    print_performance(list_scores,list_index_pmcs)
    print_selected_pmcs(my_dict, num_pmcs)
    print("\nTime(minutes) {0:.2f} ".format(round((timeit.default_timer() - start))/60,2))



Result of 4l -  KFold 5 (score_mean)


Mean Accuracy: 0.76  ['00', '27', '13']
Mean Accuracy: 0.80  ['00', '06', '12', '23', '20']
Mean Accuracy: 0.75  ['00', '21', '20', '17']
Mean Accuracy: 0.82  ['00', '27', '10', '12', '25']
Mean Accuracy: 0.78  ['00', '26', '27', '16']
Mean Accuracy: 0.78  ['00', '25', '19', '32', '15']
Mean Accuracy: 0.80  ['00', '07', '31', '27', '16']
Mean Accuracy: 0.80  ['00', '27', '13', '02', '25']
Mean Accuracy: 0.79  ['00', '27', '16', '13', '17']
Mean Accuracy: 0.78  ['00', '27', '12', '13', '09']
Mean Accuracy: 0.76  ['00', '13', '17', '27', '31']
Mean Accuracy: 0.78  ['00', '26', '21', '19', '31']
Mean Accuracy: 0.79  ['00', '26', '09', '19', '12']
Mean Accuracy: 0.81  ['00', '27', '12', '13', '25']
Mean Accuracy: 0.81  ['00', '21', '09', '13', '27']
Mean Accuracy: 0.79  ['00', '27', '13', '11', '10']
Mean Accuracy: 0.78  ['00', '27', '31', '30', '10']
Mean Accuracy: 0.79  ['00', '27', '13', '07']
Mean Accuracy: 0.80  ['00', '27', '32', '31', '10']
M

# Test Predictor


Before predict using the pmcs selected, we need to plot the pmcs to see if exist anyone empty or constant.

In [0]:
df = pd.read_csv("/mntDrive/My Drive/Doutorado/Feature Selection/data/power/NOT_normalized/4b.csv")     
plt.figure(figsize=(15,5))

df[['BR_PRED:0x12','L1D_TLB_REFILL_ST:0x4D','L2D_CACHE_LD:0x50','BR_RETURN_SPEC:0x79','BUS_ACCESS:0x19','L2D_CACHE_WB_VICTIM:0x56']].plot()
    #df[['L1D_TLB_REFILL_ST:0x4D']].plot()
plt.legend(loc='upper right')

In [0]:
df = pd.read_csv("/mntDrive/My Drive/Doutorado/Feature Selection/data/bots/NOT_normalized/4b.csv")     
plt.figure(figsize=(15,5))

df[['UNALIGNED_ST_SPEC:0x69','LDREX_SPEC:0x6C','BUS_ACCESS_NORMAL:0x64','L1D_CACHE_WB_CLEAN:0x47','MEM_ACCESS_ST:0x67','DMB_SPEC:0x7E']].plot()
plt.legend(loc='upper right')

In [0]:
print("Performance")
for i in range(5):
    df = pd.read_csv("/mntDrive/My Drive/Doutorado/Feature Selection/data/bots/NOT_normalized/4l.csv")     
    X = df[['cycles:0x08','proc_return:0x0E','reserved:0xC6','imed_branch_exec:0x0D','bus_access_read:0x60']].values
    Y = df[['speedup4l','speedup4b','speedup4b4l ']].values

    model = RandomForestRegressor(n_estimators=50, max_depth=30)
    model.fit(X, Y)
    y_pred = model.predict(X)
    print(r2_score(Y,y_pred))

print("Energy")    
for i in range(5):
    new_df = pd.read_csv("/mntDrive/My Drive/Doutorado/Feature Selection/data/power/NOT_normalized/4l.csv")
    for i in new_df.columns:
       df=remove_outlier(new_df, i)
    X = df[['cycles:0x08','proc_return:0x0E','reserved:0xC6','imed_branch_exec:0x0D','bus_access_read:0x60']].values
    Y = df[['power']].values
    
    model = RandomForestRegressor(n_estimators=50, max_depth=30)
    model.fit(X, Y.ravel())
    y_pred = model.predict(X)
    print(r2_score(Y,y_pred))



In [0]:
print("Performance")
for i in range(5):
    df = pd.read_csv("/mntDrive/My Drive/Doutorado/Feature Selection/data/bots/NOT_normalized/4b.csv")     
    X = df[['CYCLES:0x08','UNALIGNED_ST_SPEC:0x69','LDREX_SPEC:0x6C','BUS_ACCESS_NORMAL:0x64','INSTR_RETIRED:0x08','L1D_CACHE_WB_VICTIM:0x46','STREX_FAIL_SPEC:0x6E']].values
    Y = df[['speedup4l','speedup4b','speedup4b4l ']].values

    model = RandomForestRegressor(n_estimators=50, max_depth=30)
    model.fit(X, Y)
    y_pred = model.predict(X)
    print(r2_score(Y,y_pred))

print("Energy")    
for i in range(5):
    new_df = pd.read_csv("/mntDrive/My Drive/Doutorado/Feature Selection/data/power/NOT_normalized/4b.csv")
    for i in new_df.columns:
       df=remove_outlier(new_df, i)
    X = df[['CYCLES:0x08','UNALIGNED_ST_SPEC:0x69','LDREX_SPEC:0x6C','BUS_ACCESS_NORMAL:0x64','INSTR_RETIRED:0x08','L1D_CACHE_WB_VICTIM:0x46','STREX_FAIL_SPEC:0x6E']].values
    Y = df[['power']].values
    
    model = RandomForestRegressor(n_estimators=50, max_depth=30)
    model.fit(X, Y.ravel())
    y_pred = model.predict(X)
    print(r2_score(Y,y_pred))



In [0]:
print("Performance")
for i in range(5):
    df = pd.read_csv("/mntDrive/My Drive/Doutorado/Feature Selection/data/bots/NOT_normalized/4b4l.csv")     
    X = df[['cycles:0x08','no_cache_ext_mem_req:0xC1','un_load_store:0x0F','br_pred:0x10','data_snooped:0xCA','CYCLES:0x08','BR_PRED:0x12','L1D_TLB_REFILL_ST:0x4D','L2D_CACHE_LD:0x50','L1D_CACHE_WB_CLEAN:0x47','MEM_ACCESS:0x13','L1D_CACHE_REFILL_LD:0x42']].values
    Y = df[['speedup4l','speedup4b','speedup4b4l ']].values

    model = RandomForestRegressor(n_estimators=50, max_depth=30)
    model.fit(X, Y)
    y_pred = model.predict(X)
    print(r2_score(Y,y_pred))

print("Energy")    
for i in range(5):
    new_df = pd.read_csv("/mntDrive/My Drive/Doutorado/Feature Selection/data/power/NOT_normalized/4b4l.csv")
    for i in new_df.columns:
       df=remove_outlier(new_df, i)
    X = df[['cycles:0x08','no_cache_ext_mem_req:0xC1','un_load_store:0x0F','br_pred:0x10','data_snooped:0xCA','CYCLES:0x08','BR_PRED:0x12','L1D_TLB_REFILL_ST:0x4D','L2D_CACHE_LD:0x50','L1D_CACHE_WB_CLEAN:0x47','MEM_ACCESS:0x13','L1D_CACHE_REFILL_LD:0x42']].values
    Y = df[['power ']].values
    
    model = RandomForestRegressor(n_estimators=50, max_depth=30)
    model.fit(X, Y.ravel())
    y_pred = model.predict(X)
    print(r2_score(Y,y_pred))



# Plots

## Plot switch config


In [0]:
import matplotlib.pyplot as plt
import numpy as np

num_apps = 12


configs = [["4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4l","4b","4b4l"],
           ["4b4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4b","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l"],
           ["4l","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b4l"],
           ["4b4l","4b","4b","4b","4b","4b4l","4b4l","4b","4b","4b","4b","4b4l","4b","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b","4b","4b4l","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b","4b4l","4b4l","4b","4b"],
           ["4b","4b","4l","4b4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4b","4b","4b","4l","4b","4b","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4l","4b","4b","4b","4b","4b4l"],
           ["4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l"],
           ["4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4l","4l","4b4l","4b4l"],
           ["4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b","4l"],
           ["4l","4l","4l","4b","4b","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b","4b4l","4b"],
           ["4b","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l","4b4l"],
           ["4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l]","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l"],
           ["4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4b4l]","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l","4b4l","4l","4l","4b4l"],
          ]

apps = ["fib", "nqueens", "health","floorplan","fft","sort","sparselu","strassen","backpropag", "heartwall","lavaMD","particle_filter"]

y_ticks_label = ['4l', '4b', '4b4l']

fig, axs = plt.subplots(num_apps)
#fig.suptitle('Sharing both axes')
fig.set_figheight(25)
fig.set_figwidth(15)

for i in range(num_apps):
   axs[i].plot(configs[i],linestyle='--')
   axs[i].set_yticklabels(y_ticks_label)
   axs[i].title.set_text(apps[i])
    
plt.subplots_adjust(hspace=0.5)
plt.show()

## Plot energy

## Plot Energy-delay product (EDP)

## Execution time and number of switches


In [0]:
import numpy as np
import matplotlib.pyplot as plt

apps = ['fib','nqueens','health','floorplan','fft','sort','sparselu','strassen','backprop','heartwall','lavaMD','particle_filter']
scheduler_type=['Default(4b4l)','Dynamic']

default = [88.53,89.34,31.79,65.66,104.89,23.49,54.33,29.33,54.66,28.53,62.61,51.92]
default_error = [11.19,3.41,0.22,9.04,0.61,0.61,1.06,1.04,0.47,0.31,0.08,1.61]


app_name_list=["Rodinia","Rodinia","Bots","Bots"]
method_list=["Filter","Wrapper","Filter","Wrapper"]
type_of_model_list=["Decision Tree","Decision Tree","Decision Tree","Decision Tree"]

model_list = [ [87.45,86.61,31.52,58.74,100.20,23.77,51.66,29.41,55.65,30.40,44.39,36.91],
               [86.97,87.01,31.66,40.58,94.45,23.72,52.45,30.18,55.96,29.93,43.68,37.29],   
               [43.61,34.16,20.10,36.78,43.64,23.80,53.49,29.61,63.46,29.82,62.79,51.65],
               [40.41,34.19,20.15,37.27,45.11,23.69,55.72,29.67,57.33,29.71,55.25,53.59]
             ]

model_error_list = [ [34.87,3.74,0.39,30.85,38.91,0.37,1.10,1.18,1.35,0.74,3.43,1.10],
                     [34.87,3.00,0.47,19.62,52.47,0.51,2.35,4.92,1.23,0.73,1.41,1.20],
                     [30.26,1.39,0.22,5.94,0.43,0.43,0.59,1.12,1.47,0.98,0.14,1.16],
                     [1.88,1.04,0.16,4.31,1.92,0.53,2.76,1.04,2.31,1.04,1.02,4.68]
                   ]

number_switch_list = [ [0.30,0.00,0.00,8.70,0.10,0.00,0.00,1.20,1.00,9.80,1.40,1.80],
                       [0.50,0.00,0.00,1.00,0.30,0.00,0.00,0.30,1.30,11.70,1.00,1.00],
                       [144.90,38.50,1.60,1.00,54.80,0.00,0.00,0.00,2.00,0.00,0.00,0.00],
                       [95.10,4.40,1.00,1.00,1.50,0.00,11.50,1.50,7.70,0.00,1.10,192.40]
                     ]


for i in range(len(app_name_list)):
    app_name=app_name_list[i]
    method=method_list[i]
    type_of_model=type_of_model_list[i]

    model = model_list[i]
    model_error = model_error_list[i]
    number_switch = number_switch_list[i]


    perc=[]
    for i in range(len(apps)):       
      temp = (1 - (model[i]/default[i]))
      perc.append(temp)


    fig = plt.gcf()
    fig.set_size_inches(11, 6) #size of plot
    x_pos = np.arange(len(apps))

    #PLOT DO GRÁFICO 1
    ax = plt.subplot(2, 1, 1)
    ax.set_ylabel('Percentage Improvement (%)')
    ax.bar(x_pos, perc, width=0.4, color='g',tick_label=apps,capsize=3)
    ax.spines['top'].set_position('zero')#remove line top
    ax.spines['bottom'].set_position('zero')#remove line bottom
    plt.xticks(rotation=90)


    #PLOT DO GRÁFICO 2
    title_name = "Execution Time for Static(4b4l) vs Dynamic" + "\nSelect Feature - "+str(method) + "Method " + "\nModel - " + str(type_of_model)+ " based on " + str(app_name) 
    plt.title(title_name,fontsize=12)
    ax = plt.subplot(2, 1, 2)
    ax.set_ylabel('Execution Time (seconds) \n 95 CI for the Mean%')
    ax.bar(x_pos, default, yerr=default_error, width=0.4, color='b',capsize=3,label="Defaul(4b4l)")
    bar_plot = ax.bar(x_pos+0.4, model, yerr=model_error, width=0.4, color='g',tick_label=apps,capsize=3, label="Dynamic")
    ax.spines['top'].set_position('zero') #remove top lines
    plt.xticks(rotation=90)
    plt.legend()

    plt.gcf().subplots_adjust(bottom=0.25)#position image inside plot
    plt.subplots_adjust(hspace=2.0)#distance between subplot
    ax.legend(bbox_to_anchor=(1.0, 1.55))#legend position 

    #ADIÇÃO DE NÚMEROS NO TOPO DO BAR PLOT
    i=0
    for idx,rect in enumerate(bar_plot):
       height = rect.get_height()
       ax.text(rect.get_x() + rect.get_width()/2., 1.17*height, '%d' % int(number_switch[i]), ha='center', va='bottom', rotation=0)
       i = i+1

    #ADIÇÃO DE TEXTO E SETAS NO PLOT
    plt.annotate("number of\n switch ", xy=(10.1, 75),xytext=(5, 163), arrowprops=dict(arrowstyle='-|>', fc="k", ec="k", lw=1.0),size=8)
    plt.annotate("number of\n switch ", xy=(5.4, 107),xytext=(5, 163), arrowprops=dict(arrowstyle='-|>', fc="k", ec="k", lw=1.0),size=8)
    plt.annotate("number of\n switch ", xy=(1.7, 105),xytext=(5, 163), arrowprops=dict(arrowstyle='-|>', fc="k", ec="k", lw=1.0),size=8)

    file_name="model_" + app_name + "_" + method + ".png"
    #plt.savefig(file_name)
    plt.show()
    plt.clf() # clear the current figure
    plt.cla() # clear the current axes
